In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/description.txt')
data

,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [5]:
def load_data(fp):
  with open(fp,'r',encoding='utf-8') as f:
    data=f.readlines()
  data=[line.strip().split(" ::: ") for line in data]
  return data

In [6]:
import pandas as pd
train_data=load_data('/content/drive/MyDrive/train_data.txt')
train_df=pd.DataFrame(train_data,columns=['ID','Title','Genre','Description'])

test_data=load_data('/content/drive/MyDrive/test_data.txt')
test_df=pd.DataFrame(test_data,columns=['ID','Title','Description'])

test_solution=load_data('/content/drive/MyDrive/test_data_solution.txt')
test_solution_df=pd.DataFrame(test_solution,columns=['ID','Title','Genre','Description'])

In [7]:
print("Train data:")
train_df

Train data:


,ID,Title,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on B...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The siste...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about gr..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and have...


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)

X_train_tfid=vectorizer.fit_transform(train_df['Description'])
X_test_tfid=vectorizer.transform(test_df['Description'])

print("Training data shape:",X_train_tfid.shape)
print("Testing data shape:",X_test_tfid.shape)

Training data shape: (54214, 10000)
Testing data shape: (54200, 10000)


In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(train_df['Genre'])
print("Unique genres in the training data:",le.classes_)

Unique genres in the training data: ['action' 'adult' 'adventure' 'animation' 'biography' 'comedy' 'crime'
 'documentary' 'drama' 'family' 'fantasy' 'game-show' 'history' 'horror'
 'music' 'musical' 'mystery' 'news' 'reality-tv' 'romance' 'sci-fi'
 'short' 'sport' 'talk-show' 'thriller' 'war' 'western']


In [10]:
from sklearn.linear_model import LogisticRegression
lr_model=LogisticRegression(max_iter=200)
lr_model.fit(X_train_tfid,y_train)

y_pred=lr_model.predict(X_test_tfid)
pred_genres=le.inverse_transform(y_pred)

test_df['Predicted genre']=pred_genres
test_df[['Title','Predicted genre']]

,Title,Predicted genre
0,Edgar's Lunch (1998),drama
1,La guerra de papá (1977),drama
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),drama
...,...,...
54195,"""Tales of Light & Dark"" (2013)",drama
54196,Der letzte Mohikaner (1965),drama
54197,Oliver Twink (2007),comedy
54198,Slipstream (1973),drama


In [11]:
test_df['Predicted genre']=pred_genres
merge_df=pd.merge(test_solution_df[['ID','Genre']],test_df[['ID','Predicted genre']],on='ID',how='left')
merge_df

,ID,Genre,Predicted genre
0,1,thriller,drama
1,2,comedy,drama
2,3,documentary,documentary
3,4,drama,drama
4,5,drama,drama
...,...,...,...
54195,54196,horror,drama
54196,54197,western,drama
54197,54198,adult,comedy
54198,54199,drama,drama


In [12]:
from sklearn.metrics import accuracy_score,classification_report
accuracy=accuracy_score(merge_df['Genre'],merge_df['Predicted genre'])
print(f"Accuracy:,{accuracy:4f}")

print("\nClassification Report:")
print(classification_report(merge_df['Genre'],merge_df['Predicted genre']))

Accuracy:,0.594539

Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      action       0.51      0.30      0.37      1314
       adult       0.65      0.24      0.35       590
   adventure       0.67      0.16      0.25       775
   animation       0.61      0.04      0.08       498
   biography       0.00      0.00      0.00       264
      comedy       0.54      0.60      0.57      7446
       crime       0.41      0.03      0.06       505
 documentary       0.68      0.87      0.76     13096
       drama       0.55      0.78      0.65     13612
      family       0.49      0.08      0.14       783
     fantasy       0.61      0.03      0.06       322
   game-show       0.90      0.49      0.64       193
     history       0.00      0.00      0.00       243
      horror       0.66      0.57      0.61      2204
       music       0.67      0.46      0.55       731
     musical       0.44      0.01      0.03       276
     mystery       0.33      0.00      0.01       318
        news       0.64    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
from sklearn.naive_bayes import MultinomialNB
nb_model=MultinomialNB()
nb_model.fit(X_train_tfid,y_train)

y_pred_nb=nb_model.predict(X_test_tfid)
pred_genres_nb=le.inverse_transform(y_pred_nb)

test_df['Predicted genre_nb']=pred_genres_nb
merge_df_nb=pd.merge(test_solution_df,test_df[['ID','Predicted genre_nb']],on='ID',)

In [14]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_nb=accuracy_score(merge_df_nb['Genre'],merge_df_nb['Predicted genre_nb'])
print(f"Accuracy:,{accuracy_nb:4f}")

print("\nClassification Report:")
print(classification_report(merge_df_nb['Genre'],merge_df_nb['Predicted genre_nb'],target_names=le.classes_))

Accuracy:,0.509244

Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      action       0.56      0.03      0.06      1314
       adult       0.46      0.02      0.04       590
   adventure       0.77      0.04      0.08       775
   animation       0.00      0.00      0.00       498
   biography       0.00      0.00      0.00       264
      comedy       0.53      0.40      0.46      7446
       crime       0.00      0.00      0.00       505
 documentary       0.56      0.89      0.69     13096
       drama       0.44      0.84      0.58     13612
      family       0.00      0.00      0.00       783
     fantasy       0.00      0.00      0.00       322
   game-show       1.00      0.02      0.04       193
     history       0.00      0.00      0.00       243
      horror       0.77      0.23      0.35      2204
       music       0.89      0.02      0.05       731
     musical       0.00      0.00      0.00       276
     mystery       0.00      0.00      0.00       318
        news       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
from sklearn.svm import LinearSVC
svm_model=LinearSVC()
svm_model.fit(X_train_tfid,y_train)


LinearSVC()

In [16]:
y_pred_svm=svm_model.predict(X_test_tfid)
pred_genres_svm=le.inverse_transform(y_pred_svm)

test_df['Predicted genre_svm']=pred_genres_svm
merge_df_svm=pd.merge(test_solution_df,test_df[['ID','Predicted genre_svm']],on='ID',)

In [17]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_svm=accuracy_score(merge_df_svm['Genre'],merge_df_svm['Predicted genre_svm'])
print(f"Accuracy:,{accuracy_svm:4f}")

print("\nClassification Report:")
print(classification_report(merge_df_svm['Genre'],merge_df_svm['Predicted genre_svm'],target_names=le.classes_))

Accuracy:,0.583321

Classification Report:
              precision    recall  f1-score   support

      action       0.41      0.33      0.36      1314
       adult       0.59      0.42      0.49       590
   adventure       0.42      0.20      0.27       775
   animation       0.33      0.15      0.20       498
   biography       0.00      0.00      0.00       264
      comedy       0.55      0.59      0.57      7446
       crime       0.21      0.07      0.10       505
 documentary       0.70      0.83      0.76     13096
       drama       0.57      0.71      0.63     13612
      family       0.33      0.15      0.21       783
     fantasy       0.25      0.08      0.12       322
   game-show       0.79      0.64      0.70       193
     history       0.18      0.02      0.03       243
      horror       0.60      0.61      0.61      2204
       music       0.59      0.50      0.54       731
     musical       0.29      0.07      0.12       276
     mystery       0.21      0.05     

In [18]:
zoner_description=[
     'A space crew travels to a distant planet.',
     'A love story between a boy and a girl.',
     'Detectives investigate a mysterious murder.',
     'Aliens attack Earth and humans fight back.'
]

In [19]:
test_data_tfidf=vectorizer.transform(zoner_description)

In [20]:
y_pred_lr=lr_model.predict(test_data_tfidf)
pred_genres_lr=le.inverse_transform(y_pred_lr)

In [21]:
y_pred_nb=nb_model.predict(test_data_tfidf)
pred_genres_nb=le.inverse_transform(y_pred_nb)

In [22]:
y_pred_svm=svm_model.predict(test_data_tfidf)
pred_genres_svm=le.inverse_transform(y_pred_svm)

In [23]:
print("Predicting genres using Logistic Regression:",pred_genres_lr)
print("Predicting genres using Naive Bayes:",pred_genres_nb)
print("Predicting genres using SVM:",pred_genres_svm)
print()
for i,message in enumerate(zoner_description):
  print(f"Story: {message}")
  print(f"Predicted genre (LR): {pred_genres_lr[i]}")

Predicting genres using Logistic Regression: ['sci-fi' 'romance' 'thriller' 'sci-fi']
Predicting genres using Naive Bayes: ['documentary' 'drama' 'thriller' 'documentary']
Predicting genres using SVM: ['sci-fi' 'drama' 'mystery' 'sci-fi']

Story: A space crew travels to a distant planet.
Predicted genre (LR): sci-fi
Story: A love story between a boy and a girl.
Predicted genre (LR): romance
Story: Detectives investigate a mysterious murder.
Predicted genre (LR): thriller
Story: Aliens attack Earth and humans fight back.
Predicted genre (LR): sci-fi
